# DATA WRANGLING
This is the data wrangling efforts put forth to collect data into data frames for analysis in accordance with the project description.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
EIA_KEY = 'API_KEY'
DEMAND_CAT_ID = '2122628'
FORECAST_CAT_ID = '2122627'
NETGEN_CAT_ID = '2122629'
SERIES_URL = 'http://api.eia.gov/series/'
CAT_URL = 'http://api.eia.gov/category/'
REGIONS = ["US48", "CAL", "CAR", "CENT", "FLA", "MIDA", "MIDW", "NE", "NY", "NW", "SE", "SW", "TEN", "TEX"]

In [2]:
def get_series_ids(id):
    """returns a list of series ids with some supplementary information from the given category id"""
    data_series = []
    params = {'api_key': EIA_KEY, 'category_id': id}
    ses = requests.get(CAT_URL, params = params)
    cat_params = {'api_key': EIA_KEY, 'category_id': ''}
    for cat in ses.json()['category']['childcategories']:
        cat_params['category_id'] = cat['category_id']
        req = requests.get(CAT_URL, params = cat_params)
        for series in req.json()['category']['childseries']:
            if series['series_id'][-1] == 'H':
                name = series['name']
                is_region = False
                if name[name.index('(')+1:name.index(')')] == 'region':
                    is_region = True
                    name = cat['name'][cat['name'].index('(')+1:cat['name'].index(')')]
                else:
                    name = name[name.index('(')+1:name.index(')')]
                data_series.append((series['series_id'], name, is_region, series['units']))
    return data_series


def get_series(target_series):
    """returns series data from given series ids and then aggregates all data 
    by date and converts the date column from the ISO standard to a python datetime"""
    result = []
    series_params = {'api_key':EIA_KEY}
    for series in target_series:
        series_params['series_id'] = series[0]
        req = requests.get(SERIES_URL, params = series_params)
        result.append(pd.DataFrame(req.json()['series'][0]['data'], columns = ['date', series[1]]))
    #df = pd.concat(result, axis = 1)
    m = map(lambda x: x.set_index('date'), result)
    df = pd.concat(m).reset_index().groupby('date').sum()
    type_dir = {}
    for col in df.columns[1:]:
        type_dir[col] = float
    df = df.astype(type_dir)
    df = df.reset_index()
    df['date'] = pd.to_datetime(df['date'])
    return df

In [3]:
demand_df = get_series(get_series_ids(DEMAND_CAT_ID))
forecast_df = get_series(get_series_ids(FORECAST_CAT_ID))
net_gen_df = get_series(get_series_ids(NETGEN_CAT_ID))

In [16]:
demand_df[['date'] + REGIONS].head()

,date,US48,CAL,CAR,CENT,FLA,MIDA,MIDW,NE,NY,NW,SE,SW,TEN,TEX
0,2015-07-01 05:00:00+00:00,162827.0,0.0,22945.0,0.0,26384.0,84024.0,0.0,12583.0,16891.0,0.0,0.0,0.0,0.0,0.0
1,2015-07-01 06:00:00+00:00,335153.0,0.0,21396.0,28985.0,24336.0,79791.0,73432.0,12349.0,16027.0,0.0,25245.0,0.0,16136.0,37456.0
2,2015-07-01 07:00:00+00:00,333837.0,0.0,20627.0,27498.0,22842.0,76760.0,70211.0,12445.0,15541.0,12775.0,24516.0,0.0,15503.0,35119.0
3,2015-07-01 08:00:00+00:00,398386.0,38210.0,20102.0,26384.0,21906.0,74931.0,68163.0,12385.0,15283.0,34914.0,23765.0,13809.0,14896.0,33638.0
4,2015-07-01 09:00:00+00:00,388954.0,35171.0,19931.0,25663.0,21615.0,74368.0,67309.0,12387.0,15304.0,33142.0,23554.0,13049.0,14663.0,32798.0


In [17]:
forecast_df[['date'] + REGIONS].head()

,date,US48,CAL,CAR,CENT,FLA,MIDA,MIDW,NE,NY,NW,SE,SW,TEN,TEX
0,2015-07-01 05:00:00+00:00,106573.0,0.0,24453.0,0.0,24781.0,29484.0,0.0,11810.0,16045.0,0.0,0.0,0.0,0.0,0.0
1,2015-07-01 06:00:00+00:00,269675.0,0.0,22731.0,28749.0,21203.0,27756.0,76958.0,11190.0,0.0,0.0,25031.0,0.0,16349.0,39708.0
2,2015-07-01 07:00:00+00:00,269773.0,0.0,21569.0,27243.0,20009.0,26643.0,74126.0,10860.0,0.0,12519.0,23845.0,0.0,15621.0,37338.0
3,2015-07-01 08:00:00+00:00,331736.0,35264.0,20898.0,26120.0,19278.0,26098.0,72102.0,10760.0,0.0,33882.0,23062.0,13512.0,15063.0,35697.0
4,2015-07-01 09:00:00+00:00,324545.0,32894.0,20621.0,25451.0,19180.0,26289.0,71300.0,11200.0,0.0,32428.0,22807.0,12822.0,14781.0,34772.0


In [18]:
net_gen_df[['date']  + REGIONS].head()

,date,US48,CAL,CAR,CENT,FLA,MIDA,MIDW,NE,NY,NW,SE,SW,TEN,TEX
0,2015-07-01 05:00:00+00:00,157438.0,0.0,22934.0,0.0,24794.0,83944.0,0.0,11322.0,14444.0,0.0,0.0,0.0,0.0,0.0
1,2015-07-01 06:00:00+00:00,328863.0,0.0,21397.0,29484.0,23033.0,80091.0,70103.0,11376.0,13809.0,0.0,25912.0,0.0,16196.0,37462.0
2,2015-07-01 07:00:00+00:00,326919.0,0.0,20598.0,28106.0,21663.0,77994.0,64905.0,11305.0,13435.0,13093.0,25200.0,0.0,15496.0,35124.0
3,2015-07-01 08:00:00+00:00,380036.0,31005.0,20068.0,26841.0,20779.0,76820.0,62094.0,11303.0,13144.0,26717.0,24580.0,18015.0,15028.0,33642.0
4,2015-07-01 09:00:00+00:00,371162.0,28760.0,19965.0,26174.0,20506.0,76268.0,60872.0,11460.0,13147.0,25288.0,24311.0,16812.0,14794.0,32805.0


In [20]:
demand_df.to_csv("data/demand.csv")
forecast_df.to_csv("data/forecast.csv")
net_gen_df.to_csv("data/netgen.csv")

In [21]:
demand_df[['date'] + REGIONS].to_csv("data/regional_demand.csv")
forecast_df[['date'] + REGIONS].to_csv("data/regional_forecast.csv")
net_gen_df[['date']  + REGIONS].to_csv("data/regional_netgen.csv")

In [23]:
demand_df.drop(REGIONS[1:], axis = 1).to_csv("data/BA_demand.csv")
forecast_df.drop(REGIONS[1:], axis = 1).to_csv("data/BA_forecast.csv")
net_gen_df.drop(REGIONS[1:], axis = 1).to_csv("data/BA_netgen.csv")

141